### BÀI TOÁN: K-means phân cụm khu vực theo (Lưu lượng giao thông, Diện tích) ###


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## 1. Chuẩn bị dữ liệu

Tạo DataFrame từ dữ liệu bảng với 8 khu vực (KV0-KV7), mỗi khu vực có 2 thuộc tính:
- **Lưu lượng giao thông**: số lượng phương tiện (đơn vị: xe)
- **Diện tích**: diện tích khu vực (đơn vị: km²)

Dữ liệu này sẽ được dùng để phân cụm các khu vực có đặc điểm tương tự nhau.

In [ ]:
# Dữ liệu từ bảng trong đề bài
data = {
    "Ma_KV": ["KV0", "KV1", "KV2", "KV3", "KV4", "KV5", "KV6", "KV7"],
    "Luu_luong": [8000, 3000, 12000, 2000, 5000, 6000, 15000, 4000],
    "Dien_tich": [5.0, 3.0, 7.0, 2.0, 5.5, 6.0, 8.0, 3.0],
}
df = pd.DataFrame(data)
print("Dữ liệu ban đầu:")
print(df)

## 2. Chuẩn hóa dữ liệu (Z-score Normalization)

### Tại sao cần chuẩn hóa?
- **Lưu lượng** có giá trị lớn (2000-15000) 
- **Diện tích** có giá trị nhỏ (2-8)
- Nếu không chuẩn hóa, thuộc tính có giá trị lớn hơn sẽ chi phối kết quả clustering

### Công thức Z-score:
$$X_{std} = \frac{X - \mu}{\sigma}$$

Sau khi chuẩn hóa, dữ liệu có mean = 0 và std = 1, giúp các thuộc tính có trọng số ngang nhau.

### Hàm giải chuẩn hóa (inverse transform):
Chuyển đổi dữ liệu đã chuẩn hóa về đơn vị gốc để dễ hiểu kết quả:
$$X = X_{std} \times \sigma + \mu$$

In [ ]:
def zscore_fit_transform(X: np.ndarray):
    """
    Chuẩn hóa Z-score: X_std = (X - mean) / std
    
    Args:
        X: mảng dữ liệu cần chuẩn hóa (n_samples, n_features)
    
    Returns:
        X_std: dữ liệu đã chuẩn hóa
        mean: giá trị trung bình của mỗi feature
        std: độ lệch chuẩn của mỗi feature
    """
    mean = X.mean(axis=0)
    std = X.std(axis=0, ddof=0)  # ddof=0 để tính population std
    # Tránh chia cho 0 nếu có cột hằng số
    std = np.where(std == 0, 1.0, std)
    X_std = (X - mean) / std
    return X_std, mean, std


def zscore_inverse_transform(X_std: np.ndarray, mean: np.ndarray, std: np.ndarray):
    """
    Giải chuẩn hóa Z-score để quay về đơn vị gốc.
    
    Args:
        X_std: dữ liệu đã chuẩn hóa
        mean: giá trị trung bình đã lưu từ quá trình chuẩn hóa
        std: độ lệch chuẩn đã lưu từ quá trình chuẩn hóa
    
    Returns:
        X: dữ liệu ở đơn vị gốc
    """
    return X_std * std + mean

## 3. Khởi tạo K-means++ (Smart Initialization)

### Vấn đề của random initialization:
- Khởi tạo ngẫu nhiên có thể cho kết quả không tối ưu
- Các centroid ban đầu có thể gần nhau → hội tụ chậm

### K-means++ Algorithm:
1. **Bước 1**: Chọn ngẫu nhiên điểm đầu tiên làm centroid đầu tiên
2. **Bước 2**: Với mỗi điểm dữ liệu, tính khoảng cách bình phương (D²) đến centroid gần nhất
3. **Bước 3**: Chọn điểm tiếp theo làm centroid mới với xác suất tỉ lệ với D²
   - Điểm xa các centroid hiện tại có xác suất cao được chọn
4. **Lặp lại** bước 2-3 cho đến khi có đủ K centroids

### Lợi ích:
- Các centroids ban đầu được phân tán tốt hơn
- Hội tụ nhanh hơn và kết quả ổn định hơn

In [ ]:
def kmeans_pp_init(X: np.ndarray, k: int, rng: np.random.Generator):
    """
    Khởi tạo tâm cụm theo thuật toán K-means++
    
    Args:
        X: dữ liệu (n_samples, n_features)
        k: số lượng clusters
        rng: random number generator
    
    Returns:
        centroids: k centroids ban đầu (k, n_features)
    """
    n = X.shape[0]
    centroids = np.empty((k, X.shape[1]), dtype=float)

    # Chọn centroid đầu tiên ngẫu nhiên
    idx0 = rng.integers(0, n)
    centroids[0] = X[idx0]

    # Tính khoảng cách bình phương đến centroid đầu tiên
    dist2 = np.sum((X - centroids[0]) ** 2, axis=1)

    # Chọn các centroid tiếp theo
    for c in range(1, k):
        # Trường hợp đặc biệt: tất cả điểm trùng nhau
        if np.all(dist2 == 0):
            idx = rng.integers(0, n)
            centroids[c] = X[idx]
            continue

        # Chọn điểm mới với xác suất tỉ lệ với D²
        probs = dist2 / dist2.sum()
        idx = rng.choice(n, p=probs)
        centroids[c] = X[idx]

        # Cập nhật khoảng cách: D² = min(D² hiện tại, D² đến centroid mới)
        new_dist2 = np.sum((X - centroids[c]) ** 2, axis=1)
        dist2 = np.minimum(dist2, new_dist2)

    return centroids

## 4. Thuật toán K-means từ đầu (From Scratch)

### Thuật toán K-means:
**Input**: Dữ liệu X và số cụm K  
**Output**: Nhãn cluster cho mỗi điểm và K centroids

**Các bước:**
1. **Initialization**: Khởi tạo K centroids (dùng K-means++)
2. **Assignment Step**: Gán mỗi điểm vào cluster có centroid gần nhất
   - Tính khoảng cách Euclidean từ mỗi điểm đến tất cả centroids
   - Gán nhãn = cluster có khoảng cách nhỏ nhất
3. **Update Step**: Cập nhật centroid = trung bình các điểm trong cluster
   $$c_j = \frac{1}{|C_j|} \sum_{x_i \in C_j} x_i$$
4. **Convergence Check**: Kiểm tra hội tụ
   - Nếu inertia thay đổi < threshold → dừng
   - Nếu chưa hội tụ → quay lại bước 2

### Xử lý cụm rỗng (Empty Cluster):
- Nếu một cluster không có điểm nào → chọn điểm xa nhất với centroid gần nhất của nó làm centroid mới

### Inertia (Within-Cluster Sum of Squares):
$$\text{Inertia} = \sum_{j=1}^{K} \sum_{x_i \in C_j} ||x_i - c_j||^2$$

### Multiple Runs (n_init):
- Chạy thuật toán nhiều lần với khởi tạo khác nhau
- Chọn kết quả có inertia nhỏ nhất (tốt nhất)

In [ ]:
def compute_inertia(X: np.ndarray, labels: np.ndarray, centroids: np.ndarray) -> float:
    """
    Tính SSE/Inertia: tổng bình phương khoảng cách từ điểm tới centroid của cụm.
    
    Args:
        X: dữ liệu (n_samples, n_features)
        labels: nhãn cluster của mỗi điểm (n_samples,)
        centroids: tâm cụm (k, n_features)
    
    Returns:
        inertia: giá trị SSE
    """
    return float(np.sum((X - centroids[labels]) ** 2))


def kmeans_fit(
    X: np.ndarray,
    k: int,
    n_init: int = 20,
    max_iter: int = 200,
    tol: float = 1e-6,
    random_state: int = 42,
):
    """
    Thuật toán K-means clustering hoàn chỉnh.
    
    Args:
        X: dữ liệu (n_samples, n_features)
        k: số lượng clusters
        n_init: số lần chạy với khởi tạo khác nhau
        max_iter: số vòng lặp tối đa cho mỗi lần chạy
        tol: ngưỡng hội tụ (nếu inertia thay đổi < tol → dừng)
        random_state: seed cho random number generator
    
    Returns:
        best_labels: nhãn cluster tốt nhất (n_samples,)
        best_centroids: centroids tốt nhất (k, n_features)
        best_inertia: inertia nhỏ nhất
        best_iters: số vòng lặp của nghiệm tốt nhất
    """
    n = X.shape[0]
    if k < 1:
        raise ValueError("k phải >= 1")
    if k > n:
        raise ValueError(f"k = {k} > số điểm n = {n}. K-means không thể chia hơn số điểm.")

    rng_master = np.random.default_rng(random_state)

    best_inertia = np.inf
    best_labels = None
    best_centroids = None
    best_iters = None

    # Chạy n_init lần với khởi tạo khác nhau
    for _ in range(n_init):
        # Tạo random generator riêng cho mỗi lần init
        seed = int(rng_master.integers(0, 10**9))
        rng = np.random.default_rng(seed)

        # Khởi tạo centroids bằng K-means++
        centroids = kmeans_pp_init(X, k, rng)

        prev_inertia = np.inf
        for it in range(1, max_iter + 1):
            # BƯỚC 1: Assignment - Gán điểm vào cluster gần nhất
            # Tính khoảng cách bình phương: (n_samples, k)
            dist2 = np.sum((X[:, None, :] - centroids[None, :, :]) ** 2, axis=2)
            labels = np.argmin(dist2, axis=1)

            # BƯỚC 2: Update - Cập nhật centroids
            new_centroids = np.zeros_like(centroids)
            for j in range(k):
                mask = labels == j
                if np.any(mask):
                    # Centroid = trung bình các điểm trong cluster
                    new_centroids[j] = X[mask].mean(axis=0)
                else:
                    # Xử lý cụm rỗng: chọn điểm xa nhất
                    far_idx = np.argmax(np.min(dist2, axis=1))
                    new_centroids[j] = X[far_idx]

            centroids = new_centroids

            # BƯỚC 3: Kiểm tra hội tụ
            inertia = compute_inertia(X, labels, centroids)
            if abs(prev_inertia - inertia) <= tol:
                break  # Đã hội tụ
            prev_inertia = inertia

        # Lưu nghiệm tốt nhất (inertia nhỏ nhất)
        if inertia < best_inertia:
            best_inertia = inertia
            best_labels = labels.copy()
            best_centroids = centroids.copy()
            best_iters = it

    return best_labels, best_centroids, best_inertia, best_iters

## 5. Phần (a): Phân cụm với K = 3

### Quy trình thực hiện:
1. **Trích xuất features**: Lấy 2 cột "Lưu lượng" và "Diện tích" từ DataFrame
2. **Chuẩn hóa**: Áp dụng Z-score normalization
3. **Chạy K-means**: 
   - K = 3 (theo yêu cầu đề bài)
   - n_init = 30 (chạy 30 lần với khởi tạo khác nhau)
   - random_state = 7 (để kết quả có thể tái tạo)
4. **Giải chuẩn hóa centroids**: Chuyển về đơn vị gốc để dễ hiểu
5. **Hiển thị kết quả**: In bảng phân cụm và thông tin centroids

In [ ]:
# Trích xuất features để clustering
features = ["Luu_luong", "Dien_tich"]
X = df[features].to_numpy(dtype=float)

# Chuẩn hóa dữ liệu (tránh "lưu lượng" lấn át "diện tích")
X_std, mean_, std_ = zscore_fit_transform(X)

# Chạy K-means với K = 3
K = 3
labels, centroids_std, inertia, n_iters = kmeans_fit(
    X_std, k=K, n_init=30, max_iter=300, tol=1e-8, random_state=7
)

# Giải chuẩn hóa centroids để xem theo đơn vị gốc
centroids_original = zscore_inverse_transform(centroids_std, mean_, std_)

# Thêm cột Cluster vào DataFrame
df_result = df.copy()
df_result["Cluster"] = labels

# In kết quả
print("=" * 70)
print("K-MEANS (K=3) - KẾT QUẢ PHÂN CỤM")
print("=" * 70)
print(df_result.sort_values("Cluster").to_string(index=False))

print("\nTâm cụm (centroid) theo đơn vị gốc [Luu_luong, Dien_tich]:")
for i, c in enumerate(centroids_original):
    print(f"  Cụm {i}: Luu_luong={c[0]:.2f}, Dien_tich={c[1]:.2f}")

print(f"\nInertia (SSE) trên dữ liệu chuẩn hóa: {inertia:.6f}")
print(f"Số vòng lặp hội tụ (nghiệm tốt nhất): {n_iters}")

## 6. Trực quan hóa kết quả phân cụm K = 3

### Biểu đồ scatter plot gồm:
- **Trục X**: Lưu lượng giao thông (đơn vị gốc)
- **Trục Y**: Diện tích (đơn vị gốc)
- **Màu sắc**: Phân biệt các clusters khác nhau
- **Dấu X lớn**: Vị trí các centroids (đã giải chuẩn hóa)
- **Nhãn**: Mã khu vực (KV0-KV7) cạnh mỗi điểm

Biểu đồ giúp quan sát:
- Các khu vực trong cùng 1 cluster gần nhau về cả lưu lượng và diện tích
- Centroids nằm ở vị trí trung tâm của mỗi cluster

In [ ]:
# Vẽ biểu đồ scatter với màu sắc theo cluster
plt.figure(figsize=(7, 5))

# Vẽ các điểm dữ liệu, màu theo cluster
plt.scatter(df["Luu_luong"], df["Dien_tich"], c=labels, s=80, cmap='viridis', alpha=0.7)

# Vẽ centroids (dấu X lớn màu đỏ)
plt.scatter(centroids_original[:, 0], centroids_original[:, 1], 
           marker="X", s=250, c='red', edgecolors='black', linewidths=2, label='Centroids')

# Thêm nhãn cho mỗi điểm
for _, row in df_result.iterrows():
    plt.text(row["Luu_luong"] + 150, row["Dien_tich"] + 0.05, 
            row["Ma_KV"], fontsize=9, ha='left')

plt.title("K-means clustering (K=3) theo Lưu lượng & Diện tích", fontsize=12, fontweight='bold')
plt.xlabel("Lưu lượng giao thông", fontsize=10)
plt.ylabel("Diện tích (km²)", fontsize=10)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 7. Phần (b): Phương pháp Elbow để xác định K tối ưu

### Phương pháp Elbow là gì?
- Chạy K-means với nhiều giá trị K khác nhau (từ 1 đến 9)
- Tính Inertia (SSE) cho mỗi giá trị K
- Vẽ biểu đồ K vs Inertia

### Quan sát biểu đồ:
- **K nhỏ**: Inertia cao (các điểm xa centroid)
- **K tăng dần**: Inertia giảm nhanh
- **"Khuỷu tay" (Elbow)**: Điểm mà tốc độ giảm inertia chậm lại đột ngột
- **K lớn**: Inertia tiếp tục giảm nhưng chậm hơn

### Cách chọn K tối ưu:
- Chọn K tại điểm "khuỷu tay" - nơi đường cong bắt đầu "nằm ngang"
- Đây là trade-off tốt giữa:
  - **Số clusters ít** (đơn giản, dễ hiểu)
  - **Inertia thấp** (chất lượng clustering tốt)

### Lưu ý:
- Với 8 điểm dữ liệu, K > 8 không hợp lệ (mỗi điểm là 1 cluster riêng)

In [ ]:
# Chạy K-means với K từ 1 đến 9
Ks = list(range(1, 10))
inertias = []

n_points = X_std.shape[0]  # Số điểm dữ liệu (8 điểm)

for k in Ks:
    if k > n_points:
        # K > số điểm → không hợp lệ, gán NaN
        inertias.append(np.nan)
        continue
    
    # Chạy K-means và lưu inertia
    _, _, sse, _ = kmeans_fit(X_std, k=k, n_init=30, max_iter=300, tol=1e-8, random_state=7)
    inertias.append(sse)

# In bảng kết quả
print("\n" + "=" * 70)
print("ELBOW METHOD (K=1→9) - SSE/INERTIA (trên dữ liệu chuẩn hóa)")
print("=" * 70)
for k, sse in zip(Ks, inertias):
    if np.isnan(sse):
        print(f"K={k}: không hợp lệ (K > số điểm)")
    else:
        print(f"K={k}: {sse:.6f}")

## 8. Vẽ biểu đồ Elbow

Biểu đồ này giúp xác định K tối ưu:
- **Trục X**: Số lượng clusters (K)
- **Trục Y**: Inertia/SSE
- **Tìm "khuỷu tay"**: Điểm mà đường cong gãy khúc rõ ràng

**Kết luận**: 
- Quan sát biểu đồ để tìm điểm elbow
- Thường K = 2 hoặc K = 3 là lựa chọn phù hợp cho bài toán này
- Tại K = 3, inertia giảm đáng kể so với K = 2
- Với K > 3, inertia giảm ít hơn → không cần thiết phải tăng độ phức tạp

In [ ]:
# Vẽ biểu đồ Elbow
plt.figure(figsize=(8, 5))
plt.plot(Ks, inertias, marker="o", linewidth=2, markersize=8, color='steelblue')

# Đánh dấu điểm K=3 (thường là elbow point)
valid_k3_idx = Ks.index(3)
plt.scatter([3], [inertias[valid_k3_idx]], color='red', s=150, zorder=5, 
           label=f'K=3 (Elbow point)', edgecolors='black', linewidths=2)

plt.title("Elbow Method: Xác định K tối ưu (K=1→9)", fontsize=12, fontweight='bold')
plt.xlabel("Số lượng Clusters (K)", fontsize=10)
plt.ylabel("SSE / Inertia (chuẩn hóa)", fontsize=10)
plt.grid(True, alpha=0.3)
plt.xticks(Ks)
plt.legend()
plt.tight_layout()
plt.show()

print("\n" + "=" * 70)
print("KẾT LUẬN:")
print("Dựa vào biểu đồ Elbow, K tối ưu nên chọn là 3.")
print("Tại K=3, đường cong có điểm 'khuỷu tay' rõ ràng.")
print("Tăng K lên cao hơn không cải thiện đáng kể chất lượng clustering.")
print("=" * 70)